In [1]:
import pandas as pd
import requests

file = "https://data.nasa.gov/docs/legacy/meteorite_landings/Meteorite_Landings.csv"
data = pd.read_csv(file)
data.head()
data.info()
data.describe()
data.shape

,name,id,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Aachen,1,Valid,L5,21.0,Fell,1880.0,50.77500,6.08333,"(50.775, 6.08333)"
1,Aarhus,2,Valid,H6,720.0,Fell,1951.0,56.18333,10.23333,"(56.18333, 10.23333)"
2,Abee,6,Valid,EH4,107000.0,Fell,1952.0,54.21667,-113.00000,"(54.21667, -113.0)"
3,Acapulco,10,Valid,Acapulcoite,1914.0,Fell,1976.0,16.88333,-99.90000,"(16.88333, -99.9)"
4,Achiras,370,Valid,L6,780.0,Fell,1902.0,-33.16667,-64.95000,"(-33.16667, -64.95)"
...,...,...,...,...,...,...,...,...,...,...
45711,Zillah 002,31356,Valid,Eucrite,172.0,Found,1990.0,29.03700,17.01850,"(29.037, 17.0185)"
45712,Zinder,30409,Valid,"Pallasite, ungrouped",46.0,Found,1999.0,13.78333,8.96667,"(13.78333, 8.96667)"
45713,Zlin,30410,Valid,H4,3.3,Found,1939.0,49.25000,17.66667,"(49.25, 17.66667)"
45714,Zubkovsky,31357,Valid,L6,2167.0,Found,2003.0,49.78917,41.50460,"(49.78917, 41.5046)"


In [2]:
URL = "https://data.nasa.gov/resource/gh4g-9sfh.json"

response = requests.get(URL)
print(response.content)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from collections import defaultdict


print("Part A: Import and inspect the data")
print("===================================\n")

all_rows=[]
# Write your code here
BASE_URL = "https://data.nasa.gov/resource/gh4g-9sfh.json"
for offset in range(0, 50000, 1000):  
#for offset in range(0, 15, 5):  
    url = f"{BASE_URL}?$limit=1000&$offset={offset}"
    print(f"Fetching records {offset}–{offset+1000}...")
    response = requests.get(url).json()
    all_rows.append(pd.DataFrame(response))
    #print(response.items())
data = pd.concat(all_rows, ignore_index=True)

print(data.head())
print(data.info())
print(data.describe())
print(data.columns)
print("")


print("Part B: Analyze the data")
print("========================\n")

# Write your code here

num_meteorites = len(data)  
heaviest_kg = data["mass (g)"].max()
oldest_year = pd.to_numeric(data["year"], errors="coerce").min()
avg_mass_g = pd.to_numeric(data["mass (g)"], errors="coerce").mean()

print(f"Number of meteorites in the dataset: {num_meteorites}")
print(f"Heaviest meteorite (in kg): {round(float(heaviest_kg), 1)}")
print(f"Year of the oldest record in the dataset: {round(float(oldest_year), 1)}")
print(f"Average meteorite weight (in g): {round(float(avg_mass_g), 2)}")

# add decades
data["year"] = pd.to_numeric(data["year"], errors="coerce")
data["decade"] = 10 * round(data["year"]/10)
print(data["decade"].unique())
# Count meteorites per decade
decade_counts = data["decade"].value_counts().sort_index()
print(decade_counts)

# Plot
plt.figure(figsize=(8,6))
plt.plot(decade_counts.index, decade_counts.values)
plt.yscale("log")   # logarithmic scale as suggested
plt.xlabel("year (grouped)")
plt.ylabel("number of meteorites found per decade")
plt.title("Number of meteorite finds over time")

# Save figure
plt.savefig("cx_out/finds_over_time.png", dpi=150)
plt.show()

print("")
print("Part C: Visualise finds in Switzerland")
print("======================================\n")

# Write your code here

# 1) Filter Swiss data
data["reclat"] = pd.to_numeric(data["reclat"], errors="coerce")
data["reclong"] = pd.to_numeric(data["reclong"], errors="coerce")

CH_MIN_LAT, CH_MAX_LAT = 45.83, 47.79
CH_MIN_LON, CH_MAX_LON = 5.95, 10.5

df_chf = df_ch = data[
    (data["reclat"] >= CH_MIN_LAT) &
    (data["reclat"] <= CH_MAX_LAT) &
    (data["reclong"] >= CH_MIN_LON) &
    (data["reclong"] <= CH_MAX_LON)
]

df_ch.reset_index(inplace = True)
print(f"Number of meteorites in CH: {len(df_ch)}")
print(df_ch[["name", "reclat", "reclong"]])

# convert to coords
from shapely.geometry import Point
list_ch_coords = [Point(lon, lat) for lat, lon in zip(df_ch["reclat"], df_ch["reclong"])]

# plot
import geopandas as gpd


# Swiss map
gdf = gpd.read_file( "./maps/swiss.shp")
gdf.crs = "EPSG:2056"
ax = gdf.plot()
ax.margins(0)
plt.axis("off")

# add meteorites
ch_dots = gpd.GeoSeries(list_ch_coords, crs="EPSG:4326")
ch_dots = ch_dots.to_crs(gdf.crs)
ch_dots.plot( ax=ax, color="orange", markersize=30)

## add names
names = (df_ch["name"] + ", " + (round(df_ch["mass (g)"]/1000,1)).astype(str) + "kg").fillna("unknown").tolist()

for x, y, name in zip(ch_dots.x, ch_dots.y, names):
    plt.text(
      x + 3000,
      y + 3000,
      name,
      fontsize=8,
      color="black"
      )

plt.savefig("cx_out/switzerland.png")

    
### Possible Extension(s): Ask for user input and/or print all meteorites above 10kg on a world map
# Write your code below
